In [1]:
import threading
import time
import cv2
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import PIL.Image
import ipywidgets
from IPython.display import display
from ultralytics import YOLO  # YOLO 라이브러리 추가
from robotpal import Robot, Camera, bgr8_to_jpeg
from robotpal.SCSCtrl import TTLServo
import os
from datetime import datetime
import ipywidgets.widgets as widgets
import easyocr

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test_12_17.pth', map_location=torch.device('cuda')))
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

model_yolo = YOLO("runs/detect/train/weights/best.pt") 

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)
[System] WebSocket 프로세서 시작


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[TCP] 클라이언트 연결됨 (('127.0.0.1', 49235))


In [2]:
# ==========================================
# 1. 초기 설정 및 로그 창(Textarea) 생성
# ==========================================
CAM_WIDTH = 816
CAM_HEIGHT = 616

# [수정됨] Output 위젯 대신 Textarea 사용 (확실한 로그 출력용)
log_widget = ipywidgets.Textarea(
    value="",
    placeholder="로그가 여기에 표시됩니다...",
    description="📝 LOG:",
    disabled=True, # 수정 불가능하게 설정
    layout=ipywidgets.Layout(width='600px', height='200px')
)

def log_print(msg):
    """텍스트 상자에 메시지를 직접 추가하는 함수"""
    timestamp = datetime.now().strftime('%H:%M:%S')
    new_line = f"[{timestamp}] {msg}\n"
    # 새로운 로그를 맨 위에 추가 (최신순)
    log_widget.value = new_line + log_widget.value 


robot = Robot()
camera = Camera.instance(width=CAM_WIDTH, height=CAM_HEIGHT)
TTLServo.servoAngleCtrl(5, -25, 1, 100)
TTLServo.servoAngleCtrl(1, 0, 1, 100)

# ==========================================
# 2. 위젯 생성 및 배치
# ==========================================
image_widget = ipywidgets.Image(format='jpeg', width=500, height=500)
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(image_widget)
display(log_widget) # <--- [수정됨] 텍스트 상자 표시


# ==========================================
# 3. 감지 스레드
# ==========================================
stop_thread = False
detection_thread = None
latest_image_lock = threading.Lock()
shared_latest_image = None

detection_result = {
    "box": None,
    "dist_cm": None,
    "detected": False,
    "text": "" 
}

REF_DISTANCE_CM = 60.0
REF_HEIGHT_PX = 110.0

def detection_worker():
    global shared_latest_image, stop_thread, detection_result

    log_print(">>> 감지 스레드 시작됨")

    while not stop_thread:
        img_input = None
        with latest_image_lock:
            if shared_latest_image is not None:
                img_input = shared_latest_image.copy()
        
        if img_input is None:
            time.sleep(0.01)
            continue

        try:
            # YOLO 추론
            results = model_yolo(img_input, verbose=False, conf=0.1) 
            
            found = False
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                    
                    h_pixel = y2 - y1
                    dist = (REF_DISTANCE_CM * REF_HEIGHT_PX) / h_pixel if h_pixel > 0 else 0

                    detection_result["box"] = (x1, y1, x2, y2)
                    detection_result["dist_cm"] = dist
                    detection_result["detected"] = True
                    found = True
                    
                    # 120cm 이내 OCR 수행
                    if dist < 120.0:
                        try:
                            h, w, _ = img_input.shape
                            x1 = max(0, x1); y1 = max(0, y1)
                            x2 = min(w, x2); y2 = min(h, y2)
                            
                            crop_img = img_input[y1:y2, x1:x2]
                            
                            if crop_img.shape[0] > 0 and crop_img.shape[1] > 0:
                                ocr_texts = reader.readtext(crop_img, detail=0)
                                if len(ocr_texts) > 0:
                                    detection_result["text"] = " ".join(ocr_texts)
                        except: pass
                    break 
                if found: break

            if not found:
                detection_result["detected"] = False
                detection_result["box"] = None
        
        except Exception as e:
            log_print(f"🔥 감지 스레드 에러: {e}")
            time.sleep(1.0)
            
        time.sleep(0.01)

# 스레드 재시작
stop_thread = True
if detection_thread is not None:
    detection_thread.join(timeout=1.0)

stop_thread = False
shared_latest_image = None
detection_result = {"box": None, "dist_cm": None, "detected": False, "text": ""}

detection_thread = threading.Thread(target=detection_worker)
detection_thread.start()

# ==========================================
# 4. 메인 실행 루프
# ==========================================
angle = 0.0
angle_last = 0.0
stop_end_time = 0.0
is_stopped = False
frame_count = 0
ignore_detection_until = 0.0 

def execute(change):
    global angle, angle_last, is_stopped, stop_end_time, frame_count
    global shared_latest_image, ignore_detection_until

    try:
        image = change['new']
        current_time = time.time()
        frame_count += 1

        with latest_image_lock:
            shared_latest_image = image

        vis_image = image.copy()
        
        cur_box = detection_result["box"]
        cur_dist = detection_result["dist_cm"]
        is_detected = detection_result["detected"]
        cur_text = detection_result["text"]

        # --- [상태 1] 정지 모드 ---
        if is_stopped:
            if current_time < stop_end_time:
                cv2.putText(vis_image, "STOP MODE", (10, 40), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

                remain_time = stop_end_time - current_time
                cv2.putText(vis_image, f"{remain_time:.1f}s", (10, 80), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
                
                if cur_box is not None:
                    x1, y1, x2, y2 = cur_box
                    cv2.rectangle(vis_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 3)
                
                robot.left_motor.value = 0.0
                robot.right_motor.value = 0.0
                image_widget.value = bgr8_to_jpeg(vis_image)
                return
            else:
                is_stopped = False
                ignore_detection_until = current_time + 3.0 
                log_print(">>> 주행 재개 (1.5초간 쿨다운)")

        # --- [상태 2] 주행 중 감지 ---
        if is_detected and cur_dist is not None:
            x1, y1, x2, y2 = cur_box
            color = (0, 255, 255) if current_time < ignore_detection_until else (0, 255, 0)
            
            cv2.rectangle(vis_image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

            # 정지 조건 (200cm 미만)
            if cur_dist < 150.0 and current_time > ignore_detection_until:
                is_stopped = True
                stop_end_time = current_time + 1.5
                
                # 정지할 때 로그 출력
                log_print(f"🛑 정지! (거리: {cur_dist:.1f}cm) / 번호판: [{cur_text}]")
                    
                robot.left_motor.value = 0.0
                robot.right_motor.value = 0.0
                image_widget.value = bgr8_to_jpeg(vis_image)
                return

        image_widget.value = bgr8_to_jpeg(vis_image)

        # --- [상태 3] 자율 주행 ---
        # 리사이즈 없이 실행 (에러 발생 시 로그 출력)
        xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
        
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0
        
        speed_slider.value = 0.32
        
        angle = np.arctan2(x, y)
        pid = angle * 0.2 + (angle - angle_last) * 0.5
        angle_last = angle
        
        steering_slider.value = pid + 0
        left_val = max(min(speed_slider.value + steering_slider.value, 1.0), -0.9)
        right_val = max(min(speed_slider.value - steering_slider.value, 1.0), -0.9)
        
        robot.left_motor.value = float(left_val)
        robot.right_motor.value = float(right_val)
        
    except Exception as e:
        robot.left_motor.value = 0.0
        robot.right_motor.value = 0.0
        log_print(f"🚨 에러 발생: {e}")
        time.sleep(1.0)

# 실행 시작
try: camera.unobserve_all()
except: pass
execute({'new': camera.value})
camera.observe(execute, names='value')

[Send Servo] ID: 5, Angle: 24.988235294117647, Speed: 20.0
[Send Servo] ID: 1, Angle: 0.0, Speed: 20.0


Image(value=b'', format='jpeg', height='500', width='500')

Textarea(value='[00:13:54] >>> OCR 모델 로딩 완료!\n[00:13:51] >>> OCR 모델 로딩 중...\n[00:13:51] >>> 시스템 초기화 시작...\n[00…

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

opening handshake failed
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\websockets\asyncio\server.py", line 356, in conn_handler
    await connection.handshake(
    ...<3 lines>...
    )
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\websockets\asyncio\server.py", line 207, in handshake
    raise self.protocol.handshake_exc
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\websockets\server.py", line 138, in accept
    ) = self.process_request(request)
        ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\websockets\server.py", line 23

[TCP] 예외 발생: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
[TCP] 연결 종료 (('127.0.0.1', 49235))


## 프로젝트 종료하기

In [ ]:
import time
camera.unobserve(execute, names='value')
time.sleep(0.1) 
robot.stop()
camera.stop()